### 平均地価との差分等を加えていく
LogRegressionPredictor...logを取ってクロスバリデーションで学習してくれるクラス  
save_data...trainの予測値と,testの予測値をいつも同じ形で保存するための関数  

In [137]:
import numpy as np
import pandas as pd
import scipy as sp
from scipy import stats
from scipy.stats import norm, skew
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path
import datetime
import seaborn as sns
sns.set()
import matplotlib

pd.set_option('max_columns',1000)
pd.set_option('max_rows',1000)


import warnings
warnings.filterwarnings('ignore')

import re
import geocoder
from geopy.distance import great_circle, vincenty
from tqdm import tqdm
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
import os
import gc
import lightgbm as lgb
from sklearn.model_selection import KFold, train_test_split
from time import time
import datetime
from script import RegressionPredictor,LogRegressionPredictor
import japanize_matplotlib
from utils import save_data
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
SEED=1234
n_splits=10
import plotly.express as px


In [138]:
train=pd.read_csv('./input/prep_train1106.csv')
test=pd.read_csv('./input/prep_test1106.csv')
y_train = train['賃料']

In [139]:
drop_col = ['id']

## 必要な特徴量に絞る
y_train = train['賃料']
y_train_log = np.log1p(y_train)
X_train = train.drop(drop_col,axis=1)
X_test = test.drop(drop_col,axis=1)

### clustering

In [140]:
# train,test =get_Kmeans(train,test)

In [141]:
# train.to_csv('./input/prep_train1105.csv',index=None)
# test.to_csv('./input/prep_test1105.csv',index=None)

In [142]:
train['地価']=train['賃料']/train['面積']

In [143]:
def get_loc_type(data,out_col='loc_type'):
    '''緯度経度による、グリッドを作成'''
    data2=data.copy()
    data2[out_col]=0
    count=0
    for i in range(1,12):
        for j in range(11):
            data2.loc[data2.query(f'139.56+{i*0.03}<loc_lon<139.56+{(i+1)*0.03}&35.53+{j*0.03}<loc_lat<35.53+{(j+i)*0.03}').index.values,out_col]=count
            print(count,end='')
            print(data2.query(f'139.56+{i*0.03}<loc_lon<139.56+{(i+1)*0.03}&35.53+{j*0.03}<loc_lat<35.53+{(j+i)*0.03}').index.shape)
            count+=1
    return data2

# X_train['loc_type']=get_loc_type(X_train)
# X_test['loc_type']=get_loc_type(X_test)

In [144]:
def get_Kmeans(X_train,X_test,n_clusters=300,cols=['loc_lat','loc_lon','築年数'],out_col='km_type_with_age'):
    std=StandardScaler()
    
    X = X_train[cols]
    X2= X_test[cols]
    Z=pd.concat([X,X2],axis=0)
    Z=std.fit_transform(Z)

    pred = KMeans(n_clusters=n_clusters).fit_predict(Z)
    X_train[out_col]=pred[:X_train.shape[0]]
    X_test[out_col]=pred[X_train.shape[0]:]
    return X_train,X_test

In [145]:
# X_train,X_test =get_Kmeans(X_train,X_test)

In [146]:
def get_k_neighbor(df, k=10, cols=['loc_lat', 'loc_lon', '築年数']):
    '''
    k近傍点を取得
    '''
    from sklearn.neighbors import KNeighborsRegressor
    from sklearn.preprocessing import minmax_scale

    df_train = df.query('id <= @id_max')
    df_test = df.query('id > @id_max')

    df_train_k = pd.DataFrame()
    df_test_k = pd.DataFrame()

    for col in cols:
        df_train_k[col] = minmax_scale(df_train[col].values)
        df_test_k[col] = minmax_scale(df_test[col].values)

    df_train_k['target'] = df_train['賃料'].values
    df_train_k['target_per'] = df_train['賃料'].values/df_train['面積'].values

    df_train_k = df_train_k.drop_duplicates(subset=cols).dropna()
    df_train_k = df_train_k[df_train_k['target_per']!=np.inf]
    df_train_k = df_train_k.reset_index(drop=True)

    n_neighbors = k # default=5
    # 距離によって重み付けするか
    weights = 'distance' # する

    knn = KNeighborsRegressor(n_neighbors=n_neighbors, weights=weights)
    knn.fit(df_train_k.drop(['target', 'target_per'], axis=1), df_train_k['target'])

    df_train_test = df[cols].copy()

    r = knn.kneighbors(df_train_test, n_neighbors, return_distance=False)
    r_ = df_train_k['target'].loc[r.flatten()].values.flatten().reshape(r.shape)
    r_per = df_train_k['target_per'].loc[r.flatten()].values.flatten().reshape(r.shape)

    res = pd.DataFrame(r_, columns=['賃料_neighbor_%s'%(i+1) for i in range(n_neighbors)], index=df_train_test.index)
    res_per = pd.DataFrame(r_per, columns=['単位面積賃料_neighbor_%s'%(i+1) for i in range(n_neighbors)], index=df_train_test.index)

    result = pd.concat([res, res_per], axis=1)
    result.index = df.index
    result['賃料_neighbor_mean'] = result[['賃料_neighbor_%s'%(i+1) for i in range(n_neighbors)]].mean(axis=1)
    result['賃料_neighbor_std'] = result[['賃料_neighbor_%s'%(i+1) for i in range(n_neighbors)]].std(axis=1)
    result['単位面積賃料_neighbor_mean'] = result[['単位面積賃料_neighbor_%s'%(i+1) for i in range(n_neighbors)]].mean(axis=1)
    result['単位面積賃料_neighbor_std'] = result[['単位面積賃料_neighbor_%s'%(i+1) for i in range(n_neighbors)]].std(axis=1)
    result['単位面積賃料_neighbor_mean'] = result[['単位面積賃料_neighbor_%s'%(i+1) for i in range(n_neighbors)]].mean(axis=1)
    result['単位面積賃料_neighbor_std'] = result[['単位面積賃料_neighbor_%s'%(i+1) for i in range(n_neighbors)]].std(axis=1)

    df = pd.concat([df, result], axis=1)

    return df

In [147]:
# import plotly.express as px

# px.set_mapbox_access_token('pk.eyJ1IjoiemVyZWJvbSIsImEiOiJjanpjY2JkZWUwMHMwM2RwNmQyanM2cHg4In0.VvajXw6UybbJhgp_-fHeYw')
# fig = px.scatter_mapbox(mini_train, lat="loc_lat", lon="loc_lon",  color="km_type_with_age", size="賃料",text='id',
#                   color_continuous_scale=px.colors.sequential.Viridis, size_max=30, zoom=10)
# fig.show()

### target/feature_encoding 


In [148]:
def feature_encoding(train, test, category_col, target_col, func_list):
    '''target_encodingを重要な列（面積など）でやる。TEと違って、test_dfに含まれる値も集計して作る'''
    data=pd.concat([train,test],axis=0).reset_index()
    
    agg_func = {target_col: func_list}
    #agg_funcでgruopby
    agg_df = data.groupby(category_col)[target_col].agg(agg_func)
    #列名作成
    agg_df.columns = [category_col + '_' + '_'.join(col).strip() for col in agg_df.columns.values]
    #元の列に集約結果をmapしその値をコピーし新規列に加え返す。
    for col in agg_df.columns.values:
        train[col] = train[category_col].map(agg_df[col]).copy()
        test[col] = test[category_col].map(agg_df[col]).copy()
    return train, test

def target_encoding(train, test, category_col, target_col, func_list):
    '''target_encodingをやる。func_listに辞書型で列と処理する関数(meanとか)を渡す'''
    
    agg_func = {target_col: func_list}
    #agg_funcでgruopby
    agg_df = train.groupby(category_col)[target_col].agg(agg_func)
    #列名作成
    agg_df.columns = [category_col + '_' + '_'.join(col).strip() for col in agg_df.columns.values]
    #元の列に集約結果をmapしその値をコピーし新規列に加え返す。
    for col in agg_df.columns.values:
        train[col] = train[category_col].map(agg_df[col]).copy()
        test[col] = test[category_col].map(agg_df[col]).copy()
    return train, test

In [149]:
category_col='km_type_with_age'
X_train['地価'] = X_train['賃料'] / X_train['面積']
target_dict = {category_col: ['mean']}

for category_col, func_list in target_dict.items():
    X_train, X_test = feature_encoding(X_train, X_test, category_col, '面積', func_list)
    X_train, X_test = feature_encoding(X_train, X_test, category_col, '築年数', func_list)
#     X_train, X_test = target_encoding(X_train, X_test, category_col, '賃料', func_list)

target_dict = {category_col: ['mean','max','min','count']}
for category_col, func_list in target_dict.items():
    X_train, X_test = target_encoding(X_train, X_test, category_col, '地価', func_list)
    
# X_train['地価x面積'] = X_train['面積'] * X_train[f'{category_col}_地価_mean']
# X_test['地価x面積'] = X_test['面積'] * X_test[f'{category_col}_地価_mean']
# X_train['地価x面積_max'] = X_train['面積'] * X_train[f'{category_col}_地価_max']
# X_test['地価x面積_max'] = X_test['面積'] * X_test[f'{category_col}_地価_max']
# X_train['地価x面積_min'] = X_train['面積'] * X_train[f'{category_col}_地価_min']
# X_test['地価x面積_min'] = X_test['面積'] * X_test[f'{category_col}_地価_min']

X_train['area_diff'] = X_train['面積'] - X_train[f'{category_col}_面積_mean']
X_train['area_ratio'] = X_train['面積'] / X_train[f'{category_col}_面積_mean']
X_test['area_diff'] = X_test['面積'] - X_test[f'{category_col}_面積_mean']
X_test['area_ratio'] = X_test['面積'] / X_test[f'{category_col}_面積_mean']

X_train['age_diff'] = X_train['築年数'] - X_train[f'{category_col}_築年数_mean']
X_train['age_ratio'] = X_train['築年数'] / X_train[f'{category_col}_築年数_mean']
X_test['age_diff'] = X_test['築年数'] - X_test[f'{category_col}_築年数_mean']
X_test['age_ratio'] = X_test['築年数'] / X_test[f'{category_col}_築年数_mean']

X_train['area/ageratio'] = X_train['area_ratio'] / X_train['age_ratio']
X_test['area/ageratio'] = X_test['area_ratio'] / X_test['age_ratio']

X_train['地価x面積'] = X_train['面積'] * X_train[f'{category_col}_地価_mean']
X_test['地価x面積'] = X_test['面積'] * X_test[f'{category_col}_地価_mean']
X_train['地価x面積/築年数ratio'] = X_train['地価x面積'] / X_train['age_ratio']
X_test['地価x面積/築年数ratio'] = X_test['地価x面積'] / X_test['age_ratio']

### 学習準備

In [150]:
lgbm_params = {
    'num_iterations': 50000,
    'learning_rate': 0.01,
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'rmse',
    'reg_lambda': 68.19849585036235,
    'reg_alpha': 0.007585940082730346,
    'colsample_bytree': 0.6,
    'subsample': 0.5,
    'max_depth': 8,
    'min_child_weight': 8,
    'lambda': 1,
    'eta': 0.1,
    'verbose': 1,
    'random_state': 0,
    'early_stopping_round': 100
    }

lgbm_log_params=lgbm_params.copy()
lgbm_log_params['metric']='mae'
lgbm_log_params['learning_rate']=0.05

lgbm_x_params=lgbm_params.copy()
lgbm_x_params['metric']='cross_entropy'
lgbm_x_params['learning_rate']=0.05

In [151]:
X_train_raw=X_train.copy()
X_test_raw=X_test.copy()

In [153]:

X_train=X_train_raw.copy()
X_test_=X_test_raw.copy()
# features =  ['面積','築年数','sta_min','center_dis','L','loc_lat','loc_lon','総階数','畳','所在階','loc_type_地価_mean',
#              '地価x面積', 'area_diff','area_ratio', 'age_diff', 'age_ratio','area/ageratio', '地価x面積/築年数ratio']

features =  ['面積','築年数','sta_min','center_dis','L','loc_lat','loc_lon','総階数','畳','所在階','km_type_with_age_地価_mean',  '地価x面積', 'area_diff',
       'area_ratio', 'age_diff', 'age_ratio',
       'area/ageratio', '地価x面積/築年数ratio','1m2当たりの価格','鑑定評価額']

X_train = X_train[features]
X_test = X_test[features]

In [161]:
save=False
if save:
    X_train['id']=train['id']
    X_test['id']=test['id']
    X_train['賃料']=y_train
    X_train.to_csv('./psuedo/X_train1107.csv',index=None)
    X_test.to_csv('./psuedo/X_test1107.csv',index=None)
    X_train=X_train.drop(columns='賃料')

In [154]:
#RegressionPredictorに渡す↓
folder=KFold(n_splits=10, shuffle=True, random_state=421)

### xent_predict

In [155]:
# xent_func= lambda x:np.log1p(x)/np.log1p(x.max())
# xent_refunc= lambda x:np.expm1(x*14.731801689838349)

# XLGBM = LogRegressionPredictor(X_train, y_train, X_test, Folder=folder, params=lgbm_x_params,
#                                       sk_model=None, n_splits=10, clf_type='lgb',func=xent_func,revert_func=xent_refunc)
# xlgboof, xlgbpreds, xlgbFIs = XLGBM.fit()
# XLGBM.plot_FI(50)
# XLGBM.plot_pred_dist()
# XLGBM.plot_scatter_oofvspred()

In [156]:
# save_data(X_train,xlgboof, xlgbpreds,rmse=XLGBM.rmse(),name='xent_tika_km_col',save_dir='./stacking')

### normal_predict

In [157]:
# LGBM = RegressionPredictor(X_train, y_train.values, X_test, Folder=folder, params=lgbm_params,
#                                       sk_model=None, n_splits=10, clf_type='lgb')
# lgboof, lgbpreds, lgbFIs = LGBM.fit()
# LGBM.plot_FI(50)
# LGBM.plot_pred_dist()
# LGBM.plot_scatter_oofvspred()

In [158]:
# save_data(X_train,lgboof, lgbpreds,rmse=LGBM.rmse(),name='tika_kmcol',save_dir='./stacking')

### log_predict
今一番精度がいいのがこれ

In [159]:
LogLGBM = LogRegressionPredictor(X_train, y_train.values, X_test, Folder=folder, params=lgbm_log_params,
                                      sk_model=None, n_splits=10, clf_type='lgb')
log_lgboof, log_lgbpreds, log_lgbFIs = LogLGBM.fit()
LogLGBM.plot_FI(50)
LogLGBM.plot_pred_dist()
LogLGBM.plot_scatter_oofvspred()

Training on fold 1
Training until validation scores don't improve for 100 rounds.
[5000]	training's l1: 0.0178436	valid_1's l1: 0.0532828
[10000]	training's l1: 0.00832152	valid_1's l1: 0.050386
[15000]	training's l1: 0.00478576	valid_1's l1: 0.0494568
[20000]	training's l1: 0.00315016	valid_1's l1: 0.0490399
[25000]	training's l1: 0.00227938	valid_1's l1: 0.0488341
[30000]	training's l1: 0.0017622	valid_1's l1: 0.0487105
Early stopping, best iteration is:
[31820]	training's l1: 0.00162695	valid_1's l1: 0.04868


KeyboardInterrupt: 

In [136]:
save_data(X_train,log_lgboof, log_lgbpreds,rmse=LogLGBM.rmse(),name='log_tika_TE2',save_dir='./stacking')

,id,target
0,31471,91953.095541
1,31472,82887.781648
2,31473,65771.808551
3,31474,52870.605180
4,31475,80493.771793


,id,target
0,1,80883.563764
1,2,66355.648045
2,3,109951.287114
3,4,158163.625907
4,5,64341.402361


In [23]:
LogLGBM = LogRegressionPredictor(X_train, y_train.values, X_test, Folder=folder, params=lgbm_log_params,
                                      sk_model=None, n_splits=10, clf_type='lgb')
log_lgboof, log_lgbpreds, log_lgbFIs = LogLGBM.fit()
LogLGBM.plot_FI(50)
LogLGBM.plot_pred_dist()
LogLGBM.plot_scatter_oofvspred()

Training on fold 1
Training until validation scores don't improve for 50 rounds.
[5000]	training's l1: 0.0195724	valid_1's l1: 0.0543793
[10000]	training's l1: 0.00944409	valid_1's l1: 0.0514929
Early stopping, best iteration is:
[12757]	training's l1: 0.00693673	valid_1's l1: 0.0508711
Training on fold 2
Training until validation scores don't improve for 50 rounds.
[5000]	training's l1: 0.0196889	valid_1's l1: 0.0518391
Early stopping, best iteration is:
[9793]	training's l1: 0.00976461	valid_1's l1: 0.0490924
Training on fold 3
Training until validation scores don't improve for 50 rounds.
[5000]	training's l1: 0.0195213	valid_1's l1: 0.0537694
[10000]	training's l1: 0.00951098	valid_1's l1: 0.0507832
[15000]	training's l1: 0.00558235	valid_1's l1: 0.0495811
[20000]	training's l1: 0.00371427	valid_1's l1: 0.0490359
[25000]	training's l1: 0.00271198	valid_1's l1: 0.0487626


KeyboardInterrupt: 

In [50]:
save_data(X_train,log_lgboof, log_lgbpreds,rmse=LogLGBM.rmse(),name='log_km_wa_locmodified',save_dir='./stacking')

,id,target
0,31471,133824.638945
1,31472,119407.469975
2,31473,76853.771758
3,31474,60324.643624
4,31475,93411.859263


,id,target
0,1,82042.775767
1,2,64746.386218
2,3,109875.632835
3,4,184227.309161
4,5,64397.159550


In [58]:
log_lgbpreds

array([133824.63894527, 119407.46997463,  76853.7717577 , ...,
       173935.06971244, 279672.83322726,  80897.27091804])

In [59]:
log_lgboof

array([ 82042.77576689,  64746.3862177 , 109875.63283494, ...,
       124632.83604373,  94636.32579863,  78596.88861252])